In [1]:
from hgcal_dev.datasets.vertex import VertexDataModule
import numpy as np
import pandas as pd
import torch
import plotly
import plotly.express as px
from pathlib import Path
from hdbscan import HDBSCAN
from hgcal_dev.metrics.instance import match_instances
from tqdm import tqdm
from sklearn.manifold import TSNE
from omegaconf import OmegaConf
import hydra
from hydra.experimental import initialize, compose

In [13]:
import os
os.getcwd()

'/home/alexjschuy/hgcal-dev/notebooks'

In [2]:
with initialize(config_path='../alexjschuy/hgcal-dev/configs'):
    cfg = compose(config_name="config.yaml", overrides=['dataset=vertex', 'train.batch_size=1', 'criterion=centroid'])
datamodule = hydra.utils.instantiate(cfg.dataset)
datamodule.prepare_data()
datamodule.setup('fit')
train_event_paths, val_event_paths, _ = datamodule.train_val_test_split(datamodule.events)
#raw_train_events = [pd.read_pickle(f) for f in tqdm(train_event_paths)]
#raw_val_events = [pd.read_pickle(f) for f in tqdm(val_event_paths)]

In [3]:
train_dataloader = datamodule.train_dataloader()
val_dataloader = datamodule.val_dataloader()

In [34]:
train_dataloader.dataset.events[0]

PosixPath('/home/alexjschuy/data/hgcal-dev/vertex/data/event_00000.pkl')

In [38]:
event = next(iter(train_dataloader))

In [16]:
raw_event = pd.read_pickle(train_event_paths[0])

In [25]:
raw_event.shape

(2426, 27)

In [39]:
event['labels'].F.shape

torch.Size([179])

In [23]:
event['labels'].F

tensor([ 2., 20.,  7.,  4., 15.,  5.,  6.,  9.,  1., 14.,  9., 27., 16.,  0.,
        16.,  2.,  9., 20., 23.,  8., 21.,  1., 26., 16., 15., 25.,  4., 28.,
         8., 10., 22.,  5., 21.,  6., 20.,  7., 11., 23., 11., 24.,  3., 12.,
        28., 22., 16.,  5.,  7.,  3., 12., 26., 21.,  7., 23., 21.,  1.,  2.,
        12., 12.,  6.,  8.,  2.,  3.,  6.,  5., 13.,  4., 10., 27.,  2., 21.,
        12., 40.,  3., 22., 13., 13.,  8., 25.,  7., 35., 39., 42., 43., 29.,
        30., 16., 27.,  0.,  4., 37.,  1., 24.,  6., 32., 20., 36., 17.,  9.,
        23.,  5., 19., 11., 28., 14., 34., 15., 18., 18., 33., 31.,  9., 13.,
        16., 21., 27., 14., 13.,  0.,  6., 13., 23.,  5.,  2., 17., 29.,  7.,
         2.,  5., 13.,  4., 16.,  5., 21.,  6.,  3., 19.,  3.,  4.,  4., 11.,
        11.,  9.,  2., 18.,  3.,  8.,  8., 14.,  5.,  5., 14.,  4.,  3., 30.,
         2., 19.,  6.,  9.,  4.,  2., 17., 17., 16., 13.,  2., 27., 11.,  3.,
         9., 12.,  7.,  9., 16.,  3.,  9.,  1.,  2.,  4.,  2.])

In [25]:
raw_event

,fUniqueID,PID,Status,IsPU,M1,M2,D1,D2,Charge,Mass,...,Phi,Rapidity,CtgTheta,D0,DZ,T,X,Y,Z,vertex_id
0,13,-11,1,0,10,-1,-1,-1,1,0.000510,...,-0.606058,-0.103421,-0.103606,-0.000000,-2.168048,-7.340560e-11,0.000000,0.000000,-2.168048,0
2,15,11,1,0,11,-1,-1,-1,-1,0.000510,...,2.541732,1.298226,1.694890,0.000000,-2.168048,-7.340560e-11,0.000000,0.000000,-2.168048,0
4,36,-211,1,0,29,-1,-1,-1,-1,0.139570,...,2.222401,4.865433,65.874763,0.000000,-2.168048,-7.340560e-11,0.000000,0.000000,-2.168048,0
5,40,-211,1,0,29,-1,-1,-1,-1,0.139570,...,-2.881799,4.488328,46.266426,0.000000,-2.168048,-7.340560e-11,0.000000,0.000000,-2.168048,0
6,43,-211,1,0,29,-1,-1,-1,-1,0.139570,...,-1.558088,3.764825,26.768908,-0.000000,-2.168048,-7.340560e-11,0.000000,0.000000,-2.168048,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5020,5156,-211,1,1,-1,-1,-1,-1,-1,0.139570,...,3.068517,-2.986667,-15.253251,0.183288,-126.882042,1.912711e-10,-1.746732,0.341771,-153.886261,2
5021,5157,-2212,1,1,-1,-1,-1,-1,-1,0.938272,...,-0.607311,-3.073599,-36.632751,-8.771982,-362.452057,2.700723e-09,17.032457,-2.218785,-903.427307,2
5022,5158,211,1,1,-1,-1,-1,-1,1,0.139570,...,1.448368,-3.514548,-42.000710,17.176119,-907.308716,2.700723e-09,17.032457,-2.218785,-903.427307,2
5057,5193,-2212,1,1,-1,-1,-1,-1,-1,0.938272,...,0.781159,5.364282,0.000000,0.000000,0.000000,1.210007e-07,187.888397,170.710388,36147.703125,2


In [ ]:
hcal_data_dir = Path('/global/cscratch1/sd/schuya/hgcal-dev/predictions/3qkvscaz')